In [32]:
import argparse
import sys, os
from gan.gan_model import cont_style_munit_enc
from models.munit.enc_dec import MunitEncoder
from parameters.dataset import DatasetParameters
from parameters.deformation import DeformationParameters
from parameters.gan import GanParameters, MunitParameters
sys.path.append(os.path.join(sys.path[0], '/home/ibespalov/unsupervised_pattern_segmentation/'))
sys.path.append(os.path.join(sys.path[0], '/home/ibespalov/unsupervised_pattern_segmentation/gans_pytorch/'))
sys.path.append(os.path.join(sys.path[0], '/home/ibespalov/unsupervised_pattern_segmentation/gans_pytorch/stylegan2'))
sys.path.append(os.path.join(sys.path[0], '/home/ibespalov/unsupervised_pattern_segmentation/gans_pytorch/gan/'))
import albumentations
from albumentations.pytorch.transforms import ToTensor as AlbToTensor
from dataset.cardio_dataset import ImageMeasureDataset
from torch.utils import data
from transforms_utils.transforms import MaskToMeasure

In [33]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [34]:
def data_sampler(dataset, shuffle, distributed):
    if distributed:
        return data.distributed.DistributedSampler(dataset, shuffle=shuffle)

    if shuffle:
        return data.RandomSampler(dataset)

    else:
        return data.SequentialSampler(dataset)
    
def sample_data(loader):
    while True:
        for batch in loader:
            yield batch

In [35]:
transform = albumentations.Compose(
    [
        albumentations.HorizontalFlip(),
        albumentations.Resize(256, 256),
        albumentations.ElasticTransform(p=0.5, alpha=100, alpha_affine=1, sigma=10),
        albumentations.ShiftScaleRotate(p=0.5, rotate_limit=10),
        albumentations.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
        AlbToTensor()
    ]
)

In [36]:
dataset = ImageMeasureDataset(
    "/raid/data/celeba",
    "/raid/data/celeba_masks",
    img_transform=transform
)

loader = data.DataLoader(
    dataset,
    batch_size=16,
    sampler=data_sampler(dataset, shuffle=True, distributed=False),
    drop_last=True,
)

In [37]:
loader = sample_data(loader)

In [38]:
test_img, test_mask = next(loader)

ref_measure = MaskToMeasure(size=256, padding=200, clusterize=True)(image=test_img, mask=test_mask)["mask"]




In [41]:
parser = argparse.ArgumentParser(
        parents=[
            DatasetParameters(),
            GanParameters(),
            DeformationParameters(),
            MunitParameters()
        ],
        # formatter_class=argparse.ArgumentDefaultsHelpFormatter
)
munit_args = parser.parse_args()

cont_style_encoder: MunitEncoder = cont_style_munit_enc(
        munit_args,
        None, # "/home/ibespalov/pomoika/munit_content_encoder15.pt",
        None  # "/home/ibespalov/pomoika/munit_style_encoder_1.pt"
    )


import torch
weights = torch.load(f"/home/ibespalov/pomoika/stylegan2_invertable_220000.pt", map_location='cpu')
cont_style_encoder.load_state_dict(weights['enc'])
cont_style_encoder = cont_style_encoder.cuda()

usage: ipykernel_launcher.py [-h] [--image_size IMAGE_SIZE]
                             [--batch_size BATCH_SIZE]
                             [--measure_size MEASURE_SIZE] [--ngf NGF]
                             [--ndf NDF] [--L1 L1] [--noise_size NOISE_SIZE]
                             [--ct CT] [--ca CA] [--cw CW]
                             [--input_dim INPUT_DIM] [--dim DIM]
                             [--style_dim STYLE_DIM]
                             [--n_downsample N_DOWNSAMPLE] [--n_res N_RES]
                             [--activ ACTIV] [--pad_type PAD_TYPE]
                             [--norm NORM] [--mlp_dim MLP_DIM]
                             [--n_layer N_LAYER] [--num_scales NUM_SCALES]
ipykernel_launcher.py: error: unrecognized arguments: -f /run/user/1002/jupyter/kernel-58741d13-1a98-4f94-a879-b55cc884ba5a.json


SystemExit: 2